In [1]:
%load_ext autoreload
%autoreload 

import numpy as np
import matplotlib.pyplot as plt
import math
import os

from ase.io import read,write
from lammps import lammps
from langevin_ExternalField import Langevin
from ase.units import kB, fs, Pascal,mol
#from  ase.calculators.lammpslib import LAMMPSlib
from  lammpslib2 import LAMMPSlib
%env ASE_LAMMPSRUN_COMMAND=mpirun -n 4 ~/lammps-stable_29Sep2021_update1/build/lmp

env: ASE_LAMMPSRUN_COMMAND=mpirun -n 4 ~/lammps-stable_29Sep2021_update1/build/lmp


In [2]:
## variables for shell scripts ##

# for I/O
#INIT_data="INIT_data_SHELL"
#INIT_dataf="INIT_dataf_SHELL"
#INIT_read_index=":"

OUT_traj="out.traj"
OUT_log="out.log"

OUT_traj_itv=100
OUT_log_itv=20

OUT_xyz="out.xyz"

# condition

INIT_steps=100

INIT_T=800
INIT_P=100000*Pascal
INIT_dt=2*fs

dt = INIT_dt

E_FIELD=0.1

## === parameters for Langevin ====

#CELL_a=15.7372
CELL_a= 3*(5.136*(1+(INIT_T-300)*1e-5))
print(CELL_a)
INIT_fric=0.01


15.485039999999998


In [3]:
IN_DATA="YSZ_setup01_3x3x3_0.08_NVT_E0_final_forLAMMPS.data"

pos = read(IN_DATA, format="lammps-data", style="charge")
#pos = read("YSZ_setup01_3x3x3_0.08_NVT_E0_final_forLAMMPS.xyz")
#vel0= pos.get_velocities()
#print(vel0[0:10])
pos.symbols[[atom.index for atom in pos if (atom.symbol == 'H')]] = 'O'
pos.symbols[[atom.index for atom in pos if (atom.symbol == 'He')]] = 'Y'
pos.symbols[[atom.index for atom in pos if (atom.symbol == 'Li')]] = 'Zr'

print(pos.get_initial_charges()[0:10])

posf = pos.get_scaled_positions()
pos.set_cell([CELL_a, CELL_a, CELL_a])
pos.set_scaled_positions(posf)

cmds = [
    "kspace_style	ewald 1.0e-4",
    "pair_style	buck/coul/long 10.0",
    "pair_coeff	1 3 1502.11 0.345 5.1",
    "pair_coeff	1 2 1366.35 0.348 19.6",
    "pair_coeff	1 1 9547.96 0.224 32.0",
    "pair_coeff	3 3 0.0 1.0 0.0",
    "pair_coeff	2 2 0.0 1.0 0.0",
    "pair_coeff	2 3 0.0 1.0 0.0",
]

header = ['units metal',
          'atom_style charge',
          'atom_modify map array sort 0 0']

info = [
    "set type 3 charge 4",
    "set type 2 charge 3",
    "set type 1 charge -2",
]

lammps = LAMMPSlib(lmpcmds=cmds, atom_types={'O':1, 'Y':2, 'Zr':3}, lammps_header=header, amendments=info, keep_alive=True)

[ 4.  4.  4.  3. -2. -2. -2. -2. -2. -2.]


In [4]:
T = INIT_T

pos.set_calculator(lammps)
efield = np.array([E_FIELD,0,0])
#efield = np.array([0,0,0])

dyn = Langevin(atoms=pos, timestep=dt, 
               temperature_K = T, efield=efield, friction=INIT_fric, trajectory = OUT_traj, loginterval=OUT_traj_itv)

#dyn = NPT(pos, dt, temperature_K = T, externalstress = P, ttime = tdamp, pfactor = pdamp, trajectory = OUT_traj, loginterval=OUT_traj_itv)
#dyn.set_mask([[1,0,0],[0,1,0],[0,0,1]])
#Header= "step etotal T e_pot stress volume density"

Header= "step etotal T e_pot stress"
print("NVT Simulation")
print(Header)

weight = np.sum(pos.get_masses())*1.673e-27

file = open(OUT_log, "w")
file.write(Header + "\n")
def print_dyn():
    #V = pos.get_volume()*1E-30 # in m^3
    #density = weight/V
    line = f"{dyn.get_number_of_steps(): >3} {pos.get_total_energy():.6f} {pos.get_temperature():.6f} {pos.get_potential_energy():.6f} {-pos.get_stress(include_ideal_gas=True)[0:3].sum()/(3.0*Pascal):.6f}"
    #line = f"{dyn.get_number_of_steps(): >3} {pos.get_total_energy():.6f} {pos.get_temperature():.6f} {pos.get_potential_energy():.6f} {-pos.get_stress(include_ideal_gas=True)[0:3].sum()/(3.0*Pascal):.6f} {V:8.6e} {density:8.6e}"

    file.write(line+"\n")
    print(line, end = "\r")
    
    
dyn.attach(print_dyn, interval=20)

steps = INIT_steps
dyn.run(steps)
    
file.close()

NVT Simulation
step etotal T e_pot stress
